In [1]:
import numpy as np

import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
import sklearn

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam

import aif360
from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

import art
import fairlearn

import copy

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
%load_ext jupyternotify
np.random.seed(1)

<IPython.core.display.Javascript object>

In [2]:
def run_classification_metrics(CM:ClassificationMetric):
    return np.array([
        round(CM.accuracy(), 4),
        round(CM.theil_index(), 4),
        round(CM.consistency()[0], 4),
        round(CM.false_positive_rate_difference(), 4),
        round(CM.false_negative_rate_difference(), 4),
        round(CM.error_rate_difference(), 4),
        round(CM.false_discovery_rate_difference(), 4),
        round(CM.false_omission_rate_difference(), 4)
    ])

In [3]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    return np.array([
        round(BLDM.statistical_parity_difference(), 4), # negative means privileged bias
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
    ])

In [4]:
def get_comparison_algo(inprocessing_algo):
    if isinstance(inprocessing_algo, inprocessing.PrejudiceRemover):
        return sklearn.linear_model.LogisticRegression()
    if isinstance(inprocessing_algo, inprocessing.GerryFairClassifier):
        return sklearn.linear_model.LinearRegression()
    if isinstance(inprocessing_algo, inprocessing.MetaFairClassifier):
        return BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
    if isinstance(inprocessing_algo, inprocessing.ExponentiatedGradientReduction):
        return sklearn.linear_model.LogisticRegression()
    if isinstance(inprocessing_algo, inprocessing.GridSearchReduction):
        return sklearn.linear_model.LogisticRegression()

In [5]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, preprocessing.DisparateImpactRemover):
        return "Disparate Impact Remover"
    if isinstance(model, preprocessing.LFR):
        return "Learning Fair Representations"
    if isinstance(model, preprocessing.OptimPreproc):
        return "Optimized Preprocessing"
    if isinstance(model, preprocessing.Reweighing):
        return "Reweighing"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "Prejudice Remover"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "Adversarial Debiasing"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ART Classifier"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "Exp Grad Reduction"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GerryFair Classifier"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GridSearch Reduction"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MetaFair Classifier"
    
    if isinstance(model, postprocessing.EqOddsPostprocessing):
        return "Eq Odds Post."
    if isinstance(model, postprocessing.CalibratedEqOddsPostprocessing):
        return
    if isinstance(model, postprocessing.RejectOptionClassification):
        return "RejectOption Classification"
    
    return "None"

In [6]:
def get_dataset_name(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        return "German Dataset"
    if isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        return "Adult Dataset"
    if isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        return "Bank Dataset"
    if isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        return "Compas Dataset"

In [7]:
def get_optim_preproc_options_dict(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        optim_options = {
            "distortion_fun": get_distortion_german,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        optim_options = {
            "distortion_fun": get_distortion_adult,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        optim_options = {
            "distortion_fun": get_distortion_bank,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        optim_options = {
            "distortion_fun": get_distortion_compas,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    return optim_options

In [8]:
def run_base(dataset_train, dataset_test, model):
    model.fit(dataset_train.features, dataset_train.labels.ravel())
    results = model.predict(dataset_test.features)
    
    if isinstance(model, sklearn.linear_model.LinearRegression):
        results = np.rint(results)
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    return np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

In [9]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier=None, 
                 preprocessing_algo=None, inprocessing_algo=None, postprocessing_algo=None):
    base = RandomForestClassifier(n_estimators=1100)
    if inprocessing_algo is not None:
        base = get_comparison_algo(inprocessing_algo)
    base.fit(dataset_train.features, dataset_train.labels.ravel())
    results = base.predict(dataset_test.features)
    if isinstance(base, sklearn.linear_model.LinearRegression):
        results = np.rint(results)
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
    df = pd.DataFrame(metrics, columns=[get_model_name(base)])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "error rate difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    
    if preprocessing_algo is not None:
        dataset_train = preprocessing_algo.fit_transform(dataset_train)
    
    if inprocessing_algo is not None:
        inprocessing_algo.fit(dataset_train)
        fair_results = inprocessing_algo.predict(dataset_test)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = fair_results.labels
        #dataset_test_pred = results
    else:
        classifier.fit(dataset_train.features, dataset_train.labels.ravel())
        fair_results = classifier.predict(dataset_test.features)
        #print(results)
        if isinstance(classifier, sklearn.linear_model.LinearRegression):
            results = np.rint(results)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = np.array([fair_results]).transpose()
    
    if postprocessing_algo is not None:
        dataset_test_pred = postprocessing_algo.fit_predict(dataset_test, dataset_test_pred)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    name = ""
    if preprocessing_algo is not None:
        name += get_model_name(preprocessing_algo) + " + "
    if inprocessing_algo is not None:
        name += get_model_name(inprocessing_algo) + " + "
    if postprocessing_algo is not None:
        name += get_model_name(postprocessing_algo)
    df[name] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
    
    return df

In [10]:
def run_inproc_algo(dataset_train, dataset_test, unprivileged_groups, privileged_groups, inprocessing_algo, ):

    base = get_comparison_algo(inprocessing_algo)

    metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, base)
    df = pd.DataFrame(metrics, columns=[get_model_name(base)])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "error rate difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    if isinstance(dataset, GermanDataset) and isinstance(inprocessing_algo, inprocessing.GridSearchReduction):
        dataset_train = dataset_train.copy()
        dataset_train.labels = dataset_train.labels%2
        dataset_test = dataset_test.copy()
        dataset_test.labels = dataset_test.labels%2
                  
    df[get_model_name(inprocessing_algo)] = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=inprocessing_algo)
    return df

## Combination Analysis

In [11]:
dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

preprocessing_algos = [preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                      preprocessing.DisparateImpactRemover(),
                      None]
inprocessing_algos = [inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.GerryFairClassifier(),
                     inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.MetaFairClassifier(),
                     inprocessing.PrejudiceRemover(),
                     None]
postprocessing_algos = [postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                       None]

for pre in preprocessing_algos:
    for inproc in inprocessing_algos:
        for post in postprocessing_algos:
            try:
                df = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                         preprocessing_algo=copy.deepcopy(pre), 
                         inprocessing_algo=copy.deepcopy(inproc),
                         postprocessing_algo=copy.deepcopy(post))
                display(df)
            except KeyboardInterrupt:
                break
            except:
                print("FAILED: " + get_model_name(pre) + ", " + get_model_name(inproc) + ", " + get_model_name(post) + " on dataset " + get_dataset_name(dataset))
    

,Logistic Regression,Reweighing + Exp Grad Reduction + Eq Odds Post.
accuracy,0.6786,0.6629
theil index,0.1952,0.1896
consistency,0.6818,0.6818
false positive rate difference,-0.4110,0.0033
false negative rate difference,0.2707,0.0030
error rate difference,-0.0042,0.0152
false discovery rate difference,-0.0542,0.0471
false omission rate difference,0.2053,-0.0434
stat parity difference,-0.1608,0.0000
priv base rate,0.6716,0.5412


,Logistic Regression,Reweighing + Exp Grad Reduction +
accuracy,0.6786,0.6694
theil index,0.1952,0.1889
consistency,0.6818,0.6818
false positive rate difference,-0.4110,-0.0142
false negative rate difference,0.2707,-0.0330
error rate difference,-0.0042,-0.0138
false discovery rate difference,-0.0542,0.0286
false omission rate difference,0.2053,-0.0860
stat parity difference,-0.1608,0.0000
priv base rate,0.6716,0.5412


,Linear Regression,Reweighing + GerryFair Classifier + Eq Odds Post.
accuracy,0.6407,0.5775
theil index,0.2143,0.0791
consistency,0.6818,0.6818
false positive rate difference,-0.3350,-0.0035
false negative rate difference,0.2450,-0.0036
error rate difference,0.0181,0.0404
false discovery rate difference,-0.0275,0.0484
false omission rate difference,0.1991,-0.0750
stat parity difference,-0.1608,0.0000
priv base rate,0.6716,0.5412


,Linear Regression,Reweighing + GerryFair Classifier +
accuracy,0.6407,0.6045
theil index,0.2143,0.1038
consistency,0.6818,0.6818
false positive rate difference,-0.3350,-0.1407
false negative rate difference,0.2450,0.0506
error rate difference,0.0181,0.0070
false discovery rate difference,-0.0275,0.0221
false omission rate difference,0.1991,-0.0573
stat parity difference,-0.1608,0.0000
priv base rate,0.6716,0.5412


,Logistic Regression,Reweighing + GridSearch Reduction + Eq Odds Post.
accuracy,0.6786,0.5154
theil index,0.1952,0.3700
consistency,0.6818,0.6818
false positive rate difference,-0.4110,0.0377
false negative rate difference,0.2707,0.0247
error rate difference,-0.0042,0.0299
false discovery rate difference,-0.0542,0.0806
false omission rate difference,0.2053,-0.0210
stat parity difference,-0.1608,0.0000
priv base rate,0.6716,0.5412


...

,Logistic Regression,GridSearch Reduction +
accuracy,0.6786,0.5954
theil index,0.1952,0.2967
consistency,0.6818,0.6818
false positive rate difference,-0.4110,-0.7007
false negative rate difference,0.2707,0.4926
error rate difference,-0.0042,-0.0026
false discovery rate difference,-0.0542,-0.0745
false omission rate difference,0.2053,0.4545
stat parity difference,-0.1608,-0.1608
priv base rate,0.6716,0.6716


FAILED: None, MetaFair Classifier, Eq Odds Post. on dataset Compas Dataset
FAILED: None, MetaFair Classifier, None on dataset Compas Dataset


,Logistic Regression,Prejudice Remover + Eq Odds Post.
accuracy,0.6786,0.6186
theil index,0.1952,0.1553
consistency,0.6818,0.6818
false positive rate difference,-0.4110,-0.0258
false negative rate difference,0.2707,-0.0172
error rate difference,-0.0042,0.0036
false discovery rate difference,-0.0542,0.0345
false omission rate difference,0.2053,-0.0909
stat parity difference,-0.1608,-0.1608
priv base rate,0.6716,0.6716


,Logistic Regression,Prejudice Remover +
accuracy,0.6786,0.3214
theil index,0.1952,0.5937
consistency,0.6818,0.6818
false positive rate difference,-0.4110,0.3945
false negative rate difference,0.2707,-0.2601
error rate difference,-0.0042,0.0042
false discovery rate difference,-0.0542,-0.1615
false omission rate difference,0.2053,0.0528
stat parity difference,-0.1608,-0.1608
priv base rate,0.6716,0.6716


FAILED: None, None, Eq Odds Post. on dataset Compas Dataset
FAILED: None, None, None on dataset Compas Dataset


# Disregard everything underneath

In [14]:
dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

preprocessing_algos = [preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                      preprocessing.DisparateImpactRemover(),
                      None]
inprocessing_algos = [inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.GerryFairClassifier(),
                     inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.MetaFairClassifier(),
                     inprocessing.PrejudiceRemover(),
                     None]
postprocessing_algos = [postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                       None]

pre = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
inproc = inprocessing.MetaFairClassifier()
post = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

df = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                     preprocessing_algo=copy.deepcopy(pre), 
                     inprocessing_algo=copy.deepcopy(inproc),
                     postprocessing_algo=copy.deepcopy(post))
display(df)


TypeError: cannot unpack non-iterable NoneType object

In [20]:
%%notify
dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
'''dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]'''
'''dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]'''
'''dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]'''
dataset_train, dataset_test = dataset.split([0.7], shuffle = True)


base_model = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
base_metrics = run_base(dataset_train, dataset_test, base_model)

df = pd.DataFrame(base_metrics, columns=["Random Forest"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]

RW = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
dataset_train = RW.fit_transform(dataset_train.copy())
print(type(dataset_train))

fair_base_model = inprocessing.MetaFairClassifier()
fair_base_model.fit(dataset_train)
results = fair_base_model.predict(dataset_test)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

#EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
#                                                     privileged_groups=privileged_groups)


#CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
CM = ClassificationMetric(dataset_test, dataset_test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
BLDM = BinaryLabelDatasetMetric(dataset_train,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
df["Reweighing + Meta + EOP"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

''''''
DIR = preprocessing.DisparateImpactRemover()
fair_dataset_train = DIR.fit_transform(dataset_train.copy())

fair_base_model = RandomForestClassifier(n_estimators=1100)
fair_base_model.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())
results = fair_base_model.predict(dataset_test.features)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)


CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
BLDM = BinaryLabelDatasetMetric(fair_dataset_train,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
df["DIR + EOP"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

''''''
optim_options = get_optim_preproc_options_dict(dataset)
OP = OptimPreproc(OptTools, optim_options,
              unprivileged_groups = unprivileged_groups,
              privileged_groups = privileged_groups)
fair_dataset_train = OP.fit_transform(dataset_train.copy())

fair_base_model = RandomForestClassifier(n_estimators=1100)
fair_base_model.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())
results = fair_base_model.predict(dataset_test.features)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)


CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
BLDM = BinaryLabelDatasetMetric(fair_dataset_train,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
df["OP + EOP"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
df

<class 'aif360.datasets.compas_dataset.CompasDataset'>


TypeError: cannot unpack non-iterable NoneType object

<IPython.core.display.Javascript object>

In [ ]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("German Dataset"))

dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Adult Dataset"))

dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Bank Dataset"))

dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Compas Dataset"))

In [106]:
tf.compat.v1.disable_eager_execution()
dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

sess1 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var1", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_1:
    AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_1, sess=sess1, debias=False)

    AD.fit(dataset_train)
    dataset_test_pred = AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    AD_metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess1.close()

metrics = AD_metrics
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]

sess2 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var2", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_2:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_2, sess=sess2)

    fair_AD.fit(dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w/o dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
sess2.close()

'''sess3 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var3", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_3:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)

    fair_AD.fit(dataset_train.copy())
    fair_dataset_train = fair_AD.transform(dataset_train)
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)
    fair_AD.fit(fair_dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess3.close()'''

df



epoch 0; iter: 0; batch classifier loss: 641.579346
epoch 1; iter: 0; batch classifier loss: 32.879578
epoch 2; iter: 0; batch classifier loss: 17.136475
epoch 3; iter: 0; batch classifier loss: 12.746425
epoch 4; iter: 0; batch classifier loss: 9.400681
epoch 5; iter: 0; batch classifier loss: 6.140807
epoch 6; iter: 0; batch classifier loss: 2.270022
epoch 7; iter: 0; batch classifier loss: 2.144179
epoch 8; iter: 0; batch classifier loss: 0.836270
epoch 9; iter: 0; batch classifier loss: 0.199547
epoch 10; iter: 0; batch classifier loss: 0.470885
epoch 11; iter: 0; batch classifier loss: 0.412202
epoch 12; iter: 0; batch classifier loss: 0.250243
epoch 13; iter: 0; batch classifier loss: 0.263953
epoch 14; iter: 0; batch classifier loss: 0.381343
epoch 15; iter: 0; batch classifier loss: 0.254617
epoch 16; iter: 0; batch classifier loss: 0.229809
epoch 17; iter: 0; batch classifier loss: 0.286453
epoch 18; iter: 0; batch classifier loss: 0.254371
epoch 19; iter: 0; batch classifier 

ValueError: None values not supported.

In [2]:
def print_fairness_metrics(CM:ClassificationMetric):
    print(f"accuracy = {round(CM.accuracy(), 4)}")
    print(f"theil index (goal:0) = {round(CM.theil_index(), 4)}")
    print(f"binary confusion matrix = {CM.binary_confusion_matrix()}")
    print(f"consistency (goal:1) = {round(CM.consistency()[0], 4)}")
    print(f"false positive rate difference (negative:privileged bias) = {round(CM.false_positive_rate_difference(), 4)}")
    print(f"false negative rate difference (negative:privileged bias) = {round(CM.false_negative_rate_difference(), 4)}")

In [41]:
def compare_fairness_metrics_as_df(CM1:ClassificationMetric, CM2:ClassificationMetric,
                                   intervention:str) -> pd.DataFrame:
    metrics = np.array([[round(CM1.accuracy(), 4), round(CM2.accuracy(), 4)],
        [round(CM1.theil_index(), 4), round(CM2.theil_index(), 4)],
        [round(CM1.consistency()[0], 4), round(CM2.consistency()[0], 4)],
        [round(CM1.false_positive_rate_difference(), 4), round(CM2.false_positive_rate_difference(), 4)],
        [round(CM1.false_negative_rate_difference(), 4), round(CM2.false_negative_rate_difference(), 4)]]
    )
    df = pd.DataFrame(metrics, columns=["no intervention", intervention])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference"]
    return df

In [9]:
def analyze_debiasing_algos(dataset, privileged_groups, unprivileged_groups, classifier, 
                             preprocessing_algo:preprocessing = None, inprocessing_algo:inprocessing = None, 
                             postprocessing_algo:postprocessing = None):
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)
    classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    results = classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM1 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)

    if preprocessing_algo is not None:
        dataset_train = preprocessing_algo.fit_transform(dataset_train)
    fair_classifier = clone(classifier)
    fair_classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    
    results = fair_classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    
    CM2 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    return compare_fairness_metrics_as_df(CM1, CM2, "preprocessing")

In [25]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

<IPython.core.display.Javascript object>

In [37]:
RF = RandomForestClassifier(n_estimators=1100)
RF.fit(dataset_train.features, dataset_train.labels.ravel())

results = RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM1 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)

RW = preprocessing.LFR(unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

fair_dataset_train = RW.fit_transform(dataset_train)

fair_RF = RandomForestClassifier(n_estimators=1100)
fair_RF.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())

results = fair_RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM2 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)